In [166]:
#!/usr/bin/env python3

# AWS Batch Cost Estimator
#
# Reads input CMDB information from a traditional data center to size and cost estimate AWS EC2 & RDS
# infrastructure.
#
# Prerequisites:
# 1. Python3
# 2. AWS Python SDK (Boto3)
# 3. Pandas Python library
# 4. json Python library
# 5. re Python libary

# Load libraries
import pandas as pd
import json
import boto3
import re

# Configuration variables
fileInput='./requirements.csv'
fileOutput='./aws_bom.csv'

# Input column mappings

# Column which indicates source cores and peak load
srcCores = 'vCPU'
srcCPUUsage = 'CPU_Usage'

# Column which indicates peak memory useage in GB
# If srcMemUsed is blank or zero, the script will use srcMemProvsioned as the target memory
srcMemProvisioned = 'Memory_GB'
srcMemUsed = 'Memory_GB'

# Columns indicating environment (dev, test, prod, etc.)
srcEnv = 'Environment'
DEV = 'Dev'
QA = 'QA'
TEST = 'Test'

# Region mappings, including using a default region if not specified as europe or asia
srcRegion='Location'
srcASIA="AP"
awsASIA="ap-northeast-2"
srcEU='EU'
awsEU='eu-central-1'
awsDFLT='us-east-1'
awsLocDFLT="US East (N. Virginia)"
awsLocEU="EU (Frankfurt)"
awsLocASIA="Asia Pacific (Seoul)"

# OS platforms for AWS.  The customer source is all over the board and requires some manual
# tweaking.  So far only coded for Windows, RHEL, and Amazon Linux (default)
srcOS='Platform'
srcOSVer='OS Ver'
awsWindows="Windows"
awsRHEL="RHEL"
awsSLES="SLES"
awsDefault="SLES"

# Compute families.  These are super families.  X and T are actually subfamilies in the pricing API
awsComputeOptimized="Compute optimized"
awsMemoryOptimized="Memory optimized"
awsGeneralPurpose="General purpose"

# Database mappings

# Column which determines if a configuration item is an RDS candidate
# Non zero value in this column indicates a database present
srcDbInstanceCount = 'RDS_Instances'

# Dataase constants
srcOracle="Oracle"
srcSQLServer="SQL "
srcDB="DB Rel/Ver"
awsOracle="Oracle"
awsSQLServer="SQL Server"
awsAurora="Aurora MySQL"

# Fixed rates for block storage
ec2EBSUnitCost=.151
rdsEBSUnitCost=.116
srcBlockStorage='Block Storage GB'

# Open input file, read into frame
print("Reading input file....")
dfCMDB = pd.read_csv(fileInput, keep_default_na=False)

#Add RDS Column
print("Determining RDS targets....")
dfCMDB['RDS']=False

# List the servers with database instances along with type, flag as targets for RDS service
dfDBServers = dfCMDB[dfCMDB[srcDbInstanceCount] != ""]
dfDBServers = dfDBServers[dfDBServers[srcDbInstanceCount] !="0"]

indexes=dfDBServers.index.tolist()

for row in indexes:
    dfCMDB.loc[row, 'RDS'] = True
    
# Calculate cores needed from peak CPU
print("Calculating target EC2 cores...")
dfCMDB['cores_calc'] = (dfCMDB[srcCores] * dfCMDB[srcCPUUsage]) + .51
dfCMDB['cores_calc']=dfCMDB['cores_calc'].round(decimals=0)

# Correct missing used memory
for index in dfCMDB.index.tolist():
    if dfCMDB.loc[index, srcMemUsed] == 0:
        dfCMDB.loc[index, srcMemUsed] = dfCMDB.loc[index, srcMemProvisioned]
        
# Create family inference column
print("Determining EC2 instance families...")
dfCMDB['calc_family'] = ""

# Make an inference for ec2 family
dfCMDB['mem_cpu_ratio'] = dfCMDB[srcMemUsed] / dfCMDB['cores_calc']

for index in dfCMDB.index.tolist():
    if ((dfCMDB.loc[index, 'cores_calc'] <= 8) and (dfCMDB.loc[index, srcMemUsed] <=32)
        and (DEV in dfCMDB.loc[index, srcEnv]
             or QA in dfCMDB.loc[index, srcEnv]
             or TEST in dfCMDB.loc[index, srcEnv])):
            dfCMDB.loc[index, 'calc_family'] = "t"
            
    elif (dfCMDB.loc[index, 'mem_cpu_ratio'] < 3.5):
             dfCMDB.loc[index, 'calc_family'] = "c"
            
    elif (dfCMDB.loc[index, 'mem_cpu_ratio'] > 4.5):
             dfCMDB.loc[index, 'calc_family'] = "r"
            
    else:
             dfCMDB.loc[index, 'calc_family'] = "m"
            

            
# Create AWS Region Column and map to source region
# ap-northeast-2
# us-east-1
# eu-central-1
dfCMDB['AWS_Region'] = ""

print ("Mapping regions...")

for index in dfCMDB.index.tolist():
    if dfCMDB.loc[index, srcRegion] == srcASIA:
        dfCMDB.loc[index, 'AWS_Region'] = awsASIA
    elif dfCMDB.loc[index, srcRegion] == srcEU:
        dfCMDB.loc[index, 'AWS_Region'] = awsEU
    else:
        dfCMDB.loc[index, 'AWS_Region'] = awsDFLT
        
# Create AWS OS column, search for key words in source os to map to EC2 platform

print ("Determining OS platforms...")
dfCMDB['AWS_OS'] = ""

# This code required manual tweaking to account for the different representations of RedHat
for index in dfCMDB.index.tolist():
    if "Windows" in dfCMDB.loc[index, srcOS]:
        dfCMDB.loc[index, 'AWS_OS'] = awsWindows
    elif "SLES" in dfCMDB.loc[index, srcOS]:
        dfCMDB.loc[index, 'AWS_OS'] = awsSLES
    elif "Linux" in dfCMDB.loc[index, srcOS]:
        if ("RHEL" in dfCMDB.loc[index, srcOSVer] or
            "Red" in dfCMDB.loc[index, srcOSVer] or
            "RED" in dfCMDB.loc[index, srcOSVer]):
                dfCMDB.loc[index, 'AWS_OS'] = awsRHEL
        else:
            dfCMDB.loc[index, 'AWS_OS'] = awsDefault
    else:
        dfCMDB.loc[index, 'AWS_OS'] = awsDefault
        
# Core matching and pricing code
# Match calculated capacity requirements to EC2 instance types
# Price resulting EC2 instance types by hour, year, and 3-year RIs

print('Pricing EC2 instances....')

# Core matching and pricing code
# Match calculated capacity requirements to EC2 instance types
# Price resulting EC2 instance types by hour, year, and 3-year RIs

#regions = {awsDFLT, awsEU, awsASIA}
regions = {awsDFLT}
families = {"m", "c", "r"}
#oses = {awsWindows, awsRHEL, awsDefault}
oses = {awsRHEL, awsSLES}

for region in regions:
    print("Region " +region)
    for os in oses:
        print("OS " + os)
        for family in families:
            print("Family " +family)
            # Lets get specific and only get the license included, no pre-installed software, current generation, etc.
            client = boto3.client('pricing')
            
            if region == awsDFLT:
                location = awsLocDFLT
            elif region == awsEU:
                location = awsLocEU
            else:
                location = awsLocASIA
            
            if family == "c":
                instanceFamily = awsComputeOptimized
            elif family == "r":
                instanceFamily = awsMemoryOptimized
            else:
                instanceFamily = awsGeneralPurpose
    
            response = client.get_products(
                Filters=[
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'location',
                        'Value': location
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'operatingSystem',
                        'Value': os
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'instanceFamily',
                        'Value': instanceFamily
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'currentGeneration',
                        'Value': 'Yes'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'licenseModel',
                        'Value': 'No License required'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'tenancy',
                        'Value': 'Shared'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'preInstalledSw',
                        'Value': 'NA'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'operatingSystem',
                        'Value': 'RHEL'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'capacitystatus',
                        'Value': 'Used'
                    }
                    
                ],
                ServiceCode='AmazonEC2',
                MaxResults=100
            )
        
            # Let's parse the JSON and get the elements we need to map to EC2 instance type
            items=response['PriceList']

            # Lets make a dataframe with the EC2 instance choices that are rhel and memory optimized
            d={'instanceType':[], 'memory':[], 'family':[], 'one_hr_rate':[], 'one_yr_rate':[], 'three_yr_rate':[], 'vcpu':[]}
            dfInstanceList=pd.DataFrame(data=d)
            index=0

            for item in items:
                skip = False
                jItem=json.loads(item)
                itemAttributes=jItem['product']['attributes']
                instanceType=itemAttributes['instanceType']
                instancefamily=instanceType[0:2]
                
                # Filter out the new m5d types
                if (instanceType[0:3] == "m5d"):
                    skip = True
                    
                if (instanceType[0:3] == "m5a"):
                    skip = True
                    
                if (instanceType[0:2] == "c4"):
                    skip = True
                    
                if (instanceType[0:3] == "c5d"):
                    skip = True
                    
                if (instanceType[0:2] == "t2"):
                    skip = True
                    
                if (instanceType[0:2] == "m4"):
                    skip = True
                    
                if (instanceType[0:2] == "t3"):
                    skip = True
                
                if (skip != True):
                    vcpu=itemAttributes['vcpu']
                    sku=jItem['product']['sku']
                    ondemandterm="JRTCKXETXF"
                    ondemandratecode="6YS6EN2CT7"
                    oneyearterm="6QCMYABX3D"
                    oneyearratecode="2TG2D8R56U"
                    threeyearterm="NQ3QZPMQV9"
                    threeyearratecode="2TG2D8R56U"
                    onehr_rate=jItem['terms']['OnDemand'][sku+"."+ondemandterm]['priceDimensions'][sku+"."+ondemandterm+"."+ondemandratecode]['pricePerUnit']['USD']
                    oneyr_rate=jItem['terms']['Reserved'][sku+"."+oneyearterm]['priceDimensions'][sku+"."+oneyearterm+"."+oneyearratecode]['pricePerUnit']['USD']
                    threeyr_rate=jItem['terms']['Reserved'][sku+"."+threeyearterm]['priceDimensions'][sku+"."+threeyearterm+"."+threeyearratecode]['pricePerUnit']['USD']
                    
                if (skip != True):
                    memoryelement=itemAttributes['memory']
                    #strip out the nasty characters from the json memory field
                    memoryelement_strip = memoryelement.replace(' GiB','')
                    memoryelement_strip = memoryelement_strip.split(".")[0]
                    memory=re.sub('[^0-9]','', memoryelement_strip)
                        
                    dfInstanceList.loc[index, 'instanceType'] = itemAttributes['instanceType']
                    dfInstanceList.loc[index, 'memory'] =  memory
                    dfInstanceList.loc[index,'family'] = family
                    dfInstanceList.loc[index, 'vcpu'] = vcpu
                    dfInstanceList.loc[index, 'one_hr_rate'] = onehr_rate
                    dfInstanceList.loc[index, 'one_yr_rate'] = oneyr_rate
                    dfInstanceList.loc[index, 'three_yr_rate'] = threeyr_rate
                    index=index+1
                        
            skip = False
            
            #Convert elements to numeric
            dfInstanceList['memory']=dfInstanceList['memory'].apply(pd.to_numeric)
            dfInstanceList['vcpu']=dfInstanceList['vcpu'].apply(pd.to_numeric)
            dfInstanceList['one_hr_rate']=dfInstanceList['one_hr_rate'].apply(pd.to_numeric)
            dfInstanceList['one_yr_rate']=dfInstanceList['one_yr_rate'].apply(pd.to_numeric)
            dfInstanceList['three_yr_rate']=dfInstanceList['three_yr_rate'].apply(pd.to_numeric)

            #If memory optimzied sort primary by memory, otherwise sort primary by CPU
            if(family == "r"):
                dfInstanceList_sorted=dfInstanceList.sort_values(['memory', 'vcpu'], ascending=[True,True])
            else:
                dfInstanceList_sorted=dfInstanceList.sort_values(['vcpu', 'memory'], ascending=[True,True])
            
            dfInstanceList_sorted=dfInstanceList_sorted.reset_index(drop=True)
            
            dfCMDB_filter = dfCMDB[(dfCMDB.calc_family == family) & (dfCMDB.AWS_OS == os) & (dfCMDB.RDS == False) & (dfCMDB.AWS_Region == region)]
            
            index=0
            
            # Map instances to EC2 instance types  

            for index in dfCMDB_filter.index.tolist():
                found=False
                instance=0

                while ((not(found)) & (instance < len(dfInstanceList_sorted))):
                    if ((dfInstanceList_sorted.loc[instance, 'memory'] >= dfCMDB.loc[index, srcMemUsed]) and (dfInstanceList_sorted.loc[instance, 'vcpu'] >= dfCMDB.loc[index, 'cores_calc'])):
                        found = True
                        dfCMDB.loc[index, 'ec2_instance_type'] = dfInstanceList_sorted.loc[instance, 'instanceType']
                        dfCMDB.loc[index, 'one_hr_rate'] = dfInstanceList_sorted.loc[instance, 'one_hr_rate']
                        dfCMDB.loc[index, 'one_yr_rate'] = dfInstanceList_sorted.loc[instance, 'one_yr_rate']
                        dfCMDB.loc[index, 'three_yr_rate'] = dfInstanceList_sorted.loc[instance, 'three_yr_rate']
                        
                    instance = instance +1
                    
                    
# Review and price RDS                    
# Map source DB to AWS_DB

dfCMDB['AWS_DB'] = "" 
dfCMDB_filterrds = dfCMDB[(dfCMDB.RDS == True)]   
   
indexes=dfCMDB_filterrds.index.tolist()
for row in indexes:
    if srcOracle in dfCMDB.loc[row, srcDB]:
        dfCMDB.loc[row, 'AWS_DB'] = awsOracle
    elif srcSQLServer in dfCMDB.loc[row, srcDB]:
        dfCMDB.loc[row, 'AWS_DB'] = awsSQLServer
    else:
        dfCMDB.loc[row, 'AWS_DB'] = awsAurora
        
# Map RDS instances
#regions = {"us-east-1", "eu-central-1", "ap-northeast-2"}
#families = {"m", "c", "r", "t"}
#dbs = {awsOracle, awsSQLServer, awsAurora}
dbs = {awsOracle, awsAurora}

print('Pricing RDS...')

for region in regions:
    print('Region ' + region)
    for db in dbs:
        print('DB ' + db)
        for family in families:
            print('Family ' + family)
            # Lets get specific and only get the license included, no pre-installed software, current generation, etc.
            client = boto3.client('pricing')
            
            if region == awsDFLT:
                location = awsLocDFLT
            elif region == awsEU:
                location = awsLocEU
            else:
                location = awsLocASIA
            
            if family == "r":
                instanceFamily = awsMemoryOptimized
            else:
                instanceFamily = awsGeneralPurpose
                
            if db == awsAurora:
                licensemodel="No license required"
                instanceFamily = awsMemoryOptimized
            else:
                licensemodel="License included"
    
            response = client.get_products(
                Filters=[
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'location',
                        'Value': location
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'databaseEngine',
                        'Value': db
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'currentGeneration',
                        'Value': 'Yes'            
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'instanceFamily',
                        'Value': instanceFamily          
                    },      
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'licenseModel',
                        'Value': licensemodel            
                    }, 
                    ],                
                    ServiceCode='AmazonRDS',
                    MaxResults=100
                )
        
            # Let's parse the JSON and get the elements we need to map to EC2 instance type
            items=response['PriceList']

            # Lets make a dataframe with the RDS instance choices
            d={'instanceType':[], 'memory':[], 'family':[], 'one_hr_rate':[], 'one_yr_rate':[], 'three_yr_rate':[], 'vcpu':[]}
            dfInstanceList=pd.DataFrame(data=d)
            index=0

            for item in items:
                jItem=json.loads(item)
                itemAttributes=jItem['product']['attributes']
                instanceType=itemAttributes['instanceType']
                instancefamily=instanceType[3:5]
                vcpu=itemAttributes['vcpu']
                sku=jItem['product']['sku']
                ondemandterm="JRTCKXETXF"
                ondemandratecode="6YS6EN2CT7"
                
                if (db == awsSQLServer):
                    oneyearterm="HU7G6KETJZ"
                else:
                    oneyearterm="6QCMYABX3D"
                
                oneyearratecode="2TG2D8R56U"
                onyearratecode="6YS6EN2CT7"
                threeyearterm="NQ3QZPMQV9"
                threeyearratecode="2TG2D8R56U"
                onehr_rate=jItem['terms']['OnDemand'][sku+"."+ondemandterm]['priceDimensions'][sku+"."+ondemandterm+"."+ondemandratecode]['pricePerUnit']['USD']
                oneyr_rate=jItem['terms']['Reserved'][sku+"."+oneyearterm]['priceDimensions'][sku+"."+oneyearterm+"."+oneyearratecode]['pricePerUnit']['USD']
                
                # Account for the absence of an 'one-year all up-front' option for SQL server
                if db == awsSQLServer:  
                    sqlhourly=float(jItem['terms']['Reserved'][sku+"."+oneyearterm]['priceDimensions'][sku+"."+oneyearterm+"."+ondemandratecode]['pricePerUnit']['USD'])
                    oneyr_rate = float(oneyr_rate) + (sqlhourly * 8760)

                threeyr_rate=jItem['terms']['Reserved'][sku+"."+threeyearterm]['priceDimensions'][sku+"."+threeyearterm+"."+threeyearratecode]['pricePerUnit']['USD']
           
                # person = input('Enter your name: ')
        
                # Load rates
                memoryelement=itemAttributes['memory']
                        
                #strip out the nasty characters from the json memory field
                memoryelement_strip = memoryelement.replace(' GiB','')
                memoryelement_strip = memoryelement_strip.split(".")[0]
                memory=re.sub('[^0-9]','', memoryelement_strip)

                dfInstanceList.loc[index, 'instanceType'] = itemAttributes['instanceType']
                dfInstanceList.loc[index, 'memory'] =  memory
                dfInstanceList.loc[index,'family'] = family
                dfInstanceList.loc[index, 'vcpu'] = vcpu 
                dfInstanceList.loc[index, 'one_hr_rate'] = onehr_rate
                dfInstanceList.loc[index, 'one_yr_rate'] = oneyr_rate
                dfInstanceList.loc[index, 'three_yr_rate'] = threeyr_rate
                index=index+1
            
            #Convert elements to numeric
            
            dfInstanceList['memory']=dfInstanceList['memory'].apply(pd.to_numeric)
            dfInstanceList['vcpu']=dfInstanceList['vcpu'].apply(pd.to_numeric)
            dfInstanceList['one_hr_rate']=dfInstanceList['one_hr_rate'].apply(pd.to_numeric)
            dfInstanceList['one_yr_rate']=dfInstanceList['one_yr_rate'].apply(pd.to_numeric)
            dfInstanceList['three_yr_rate']=dfInstanceList['three_yr_rate'].apply(pd.to_numeric)

            # If family is memory optimized, sorty primarily by memory, otherwise primarly cpu
            if(family == "r"):
                dfInstanceList_sorted=dfInstanceList.sort_values(['memory', 'vcpu'], ascending=[True,True])
            else:
                dfInstanceList_sorted=dfInstanceList.sort_values(['vcpu', 'memory'], ascending=[True,True])

            dfInstanceList_sorted=dfInstanceList_sorted.reset_index(drop=True)
            
            myregion=region
                    
            dfCMDB_filter = dfCMDB[(dfCMDB.calc_family == family) & (dfCMDB.AWS_DB == db) & (dfCMDB.RDS == True) & (dfCMDB.AWS_Region == myregion)]

            
            # Map instances to EC2 instance types  
            for index in dfCMDB_filter.index.tolist():
                found=False
                instance=0
                
                while ((not(found)) & (instance < len(dfInstanceList_sorted))):
                    if ((dfInstanceList_sorted.loc[instance, 'memory'] >= dfCMDB.loc[index, 'Peak Mem Used']) and (dfInstanceList_sorted.loc[instance, 'vcpu'] >= dfCMDB.loc[index, 'cores_calc'])):
                        found = True
                        dfCMDB.loc[index, 'ec2_instance_type'] = dfInstanceList_sorted.loc[instance, 'instanceType']
                        dfCMDB.loc[index, 'one_hr_rate'] = dfInstanceList_sorted.loc[instance, 'one_hr_rate']
                        dfCMDB.loc[index, 'one_yr_rate'] = dfInstanceList_sorted.loc[instance, 'one_yr_rate']
                        dfCMDB.loc[index, 'three_yr_rate'] = dfInstanceList_sorted.loc[instance, 'three_yr_rate']

                    instance = instance +1
                    
# Compute EBS and snapshots
dfCMDB['ebs_month_rate'] = 0
print('Pricing EBS and snapshots...')

# Flat storage rates assuming 1% monthly rate of change on EC2

for index in dfCMDB.index.tolist():
    if dfCMDB.loc[index, 'RDS'] == True:
        dfCMDB.loc[index, 'ebs_month_rate'] = dfCMDB.loc[index, srcBlockStorage] * rdsEBSUnitCost
    else:
        dfCMDB.loc[index, 'ebs_month_rate'] = dfCMDB.loc[index, srcBlockStorage] * ec2EBSUnitCost

# Write output file
print("Writing output file...")
dfCMDB.to_csv(fileOutput)


Reading input file....
Determining RDS targets....
Calculating target EC2 cores...
Determining EC2 instance families...
Mapping regions...
Determining OS platforms...
Pricing EC2 instances....
Region us-east-1
OS SLES
Family m
Family r
Family c
OS RHEL
Family m
Family r
Family c
Pricing RDS...
Region us-east-1
DB Aurora MySQL
Family m
Family r
Family c
DB Oracle
Family m
Family r
Family c
Pricing EBS and snapshots...
Writing output file...


In [108]:
print (jItem)

{'product': {'productFamily': 'Compute Instance', 'attributes': {'enhancedNetworkingSupported': 'Yes', 'memory': '64 GiB', 'dedicatedEbsThroughput': '2120 Mbps', 'vcpu': '16', 'capacitystatus': 'UnusedCapacityReservation', 'locationType': 'AWS Region', 'storage': 'EBS only', 'instanceFamily': 'General purpose', 'operatingSystem': 'RHEL', 'physicalProcessor': 'Intel Xeon Platinum 8175', 'clockSpeed': '2.5 GHz', 'ecu': '60', 'networkPerformance': 'Up to 10 Gigabit', 'servicename': 'Amazon Elastic Compute Cloud', 'instancesku': '3PFRBSXSD3ASHUXZ', 'instanceType': 'm5.4xlarge', 'tenancy': 'Shared', 'usagetype': 'UnusedBox:m5.4xlarge', 'normalizationSizeFactor': '32', 'processorFeatures': 'Intel AVX, Intel AVX2, Intel AVX512, Intel Turbo', 'servicecode': 'AmazonEC2', 'licenseModel': 'No License required', 'currentGeneration': 'Yes', 'preInstalledSw': 'NA', 'location': 'US East (N. Virginia)', 'processorArchitecture': '64-bit', 'operation': 'RunInstances:0010'}, 'sku': '6ZPSRWHN5YSWZ6EJ'}, '

In [109]:
jItem['terms']

{'OnDemand': {'6ZPSRWHN5YSWZ6EJ.JRTCKXETXF': {'effectiveDate': '2018-11-01T00:00:00Z',
   'offerTermCode': 'JRTCKXETXF',
   'priceDimensions': {'6ZPSRWHN5YSWZ6EJ.JRTCKXETXF.6YS6EN2CT7': {'appliesTo': [],
     'beginRange': '0',
     'description': '$0.898 per Unused Reservation RHEL m5.4xlarge Instance Hour',
     'endRange': 'Inf',
     'pricePerUnit': {'USD': '0.8980000000'},
     'rateCode': '6ZPSRWHN5YSWZ6EJ.JRTCKXETXF.6YS6EN2CT7',
     'unit': 'Hrs'}},
   'sku': '6ZPSRWHN5YSWZ6EJ',
   'termAttributes': {}}}}

In [110]:
print(instancefamily)

m5


In [111]:
print(instanceType)

m5.4xlarge


In [104]:
print(family)

m


In [105]:
print(os)

RHEL


In [106]:
print(region)

us-east-1


In [47]:
dfCMDB

,Server Name,Role,Total SAP VG in MB,CPU,Memory_GB,vCPU,OS,OS Ver,Block Storage GB,CPU_Usage,Environment,Location,RDS_Instances,Platform,RDS,cores_calc,calc_family,mem_cpu_ratio,AWS_Region,AWS_OS
0,ODI005BPDP01.ODDC.CHRYSLER.COM,DB2 Database,15245568,24,104.0,19,SLES,,15246,1,Not Given,Not Given,,Linux,False,20.0,r,5.200000,us-east-1,Linux
1,ODI008EPAP01.ODDC.CHRYSLER.COM,App Server,275968,60,128.0,47,SLES,,276,1,Not Given,Not Given,,Linux,False,48.0,c,2.666667,us-east-1,Linux
2,ODI005EPDP01.ODDC.CHRYSLER.COM,App Server,15117056,24,128.0,19,SLES,,15118,1,Not Given,Not Given,,Linux,False,20.0,r,6.400000,us-east-1,Linux
3,ODI005PPDP01.ODDC.CHRYSLER.COM,DB2 Database,10407296,16,61.4,13,SLES,,10408,1,Not Given,Not Given,,Linux,False,14.0,m,4.385714,us-east-1,Linux
4,ODI005SPAP01.ODDC.CHRYSLER.COM,App Server,6208256,16,20.4,13,SLES,,6209,1,Not Given,Not Given,,Linux,False,14.0,c,1.457143,us-east-1,Linux
5,SHI006BPAP01.SHDC.CHRYSLER.COM,App Server,2643456,20,65.5,16,SLES,,2644,1,Not Given,Not Given,,Linux,False,17.0,m,3.852941,us-east-1,Linux
6,SHI008BPDP01.SHDC.CHRYSLER.COM,DB2 Database,74022272,32,264.0,25,SLES,,74023,1,Not Given,Not Given,,Linux,False,26.0,r,10.153846,us-east-1,Linux
7,SHI008EPDP01.SHDC.CHRYSLER.COM,DB2 Database,13979136,24,374.0,19,SLES,,13980,1,Not Given,Not Given,,Linux,False,20.0,r,18.700000,us-east-1,Linux
8,SHI008PPDP01.SHDC.CHRYSLER.COM,DB2 Database,11079168,16,65.5,13,SLES,,11080,1,Not Given,Not Given,,Linux,False,14.0,r,4.678571,us-east-1,Linux
9,SHI006MGRP01.SHDC.CHRYSLER.COM,App Server /DB2 Database,2244032,8,16.3,7,SLES,,2245,1,Not Given,Not Given,,Linux,False,8.0,c,2.037500,us-east-1,Linux


In [142]:
dfCMDB_filter

,Server Name,Role,Total SAP VG in MB,CPU,Memory_GB,vCPU,OS,OS Ver,Block Storage GB,CPU_Usage,Environment,Location,RDS_Instances,Platform,RDS,cores_calc,calc_family,mem_cpu_ratio,AWS_Region,AWS_OS
3,ODI005PPDP01.ODDC.CHRYSLER.COM,DB2 Database,10407296,16,61.4,13,SLES,,10408,1,Not Given,Not Given,,Linux,False,14.0,m,4.385714,us-east-1,SLES
5,SHI006BPAP01.SHDC.CHRYSLER.COM,App Server,2643456,20,65.5,16,SLES,,2644,1,Not Given,Not Given,,Linux,False,17.0,m,3.852941,us-east-1,SLES
32,shi010epas01_PE3_01,Application Server,,16,66.0,16,SLES,,60,1,Not Given,Not Given,,Linux,False,17.0,m,3.882353,us-east-1,SLES
50,shi010epas01_RRU_00,Application Server,,16,66.0,16,SLES,,60,1,Not Given,Not Given,,Linux,False,17.0,m,3.882353,us-east-1,SLES


In [143]:
dfInstanceList_sorted

,family,instanceType,memory,one_hr_rate,one_yr_rate,three_yr_rate,vcpu


In [18]:
dfInstanceList

,family,instanceType,memory,one_hr_rate,one_yr_rate,three_yr_rate,vcpu


In [29]:
print(instanceFamily)

General purpose


In [30]:
print(family)

m


In [32]:
print(skip)

False


In [34]:
print(location)

US East (N. Virginia)


In [96]:
print(items)

['{"product":{"productFamily":"Compute Instance","attributes":{"memory":"8 GiB","vcpu":"2","capacitystatus":"UnusedCapacityReservation","locationType":"AWS Region","storage":"EBS only","instanceFamily":"General purpose","operatingSystem":"RHEL","physicalProcessor":"Intel Xeon Family","clockSpeed":"Up to 3.0 GHz","ecu":"Variable","networkPerformance":"Low to Moderate","servicename":"Amazon Elastic Compute Cloud","instancesku":"CE5UXUUUXVX4BZXY","instanceType":"t2.large","tenancy":"Shared","usagetype":"UnusedBox:t2.large","normalizationSizeFactor":"4","processorFeatures":"Intel AVX; Intel Turbo","servicecode":"AmazonEC2","licenseModel":"No License required","currentGeneration":"Yes","preInstalledSw":"NA","location":"US East (N. Virginia)","processorArchitecture":"64-bit","operation":"RunInstances:0010"},"sku":"24F2DUXJD4NRSAVM"},"serviceCode":"AmazonEC2","terms":{"OnDemand":{"24F2DUXJD4NRSAVM.JRTCKXETXF":{"priceDimensions":{"24F2DUXJD4NRSAVM.JRTCKXETXF.6YS6EN2CT7":{"unit":"Hrs","endRange

In [95]:
print(response)

{'FormatVersion': 'aws_v1', 'PriceList': ['{"product":{"productFamily":"Compute Instance","attributes":{"memory":"8 GiB","vcpu":"2","capacitystatus":"UnusedCapacityReservation","locationType":"AWS Region","storage":"EBS only","instanceFamily":"General purpose","operatingSystem":"RHEL","physicalProcessor":"Intel Xeon Family","clockSpeed":"Up to 3.0 GHz","ecu":"Variable","networkPerformance":"Low to Moderate","servicename":"Amazon Elastic Compute Cloud","instancesku":"CE5UXUUUXVX4BZXY","instanceType":"t2.large","tenancy":"Shared","usagetype":"UnusedBox:t2.large","normalizationSizeFactor":"4","processorFeatures":"Intel AVX; Intel Turbo","servicecode":"AmazonEC2","licenseModel":"No License required","currentGeneration":"Yes","preInstalledSw":"NA","location":"US East (N. Virginia)","processorArchitecture":"64-bit","operation":"RunInstances:0010"},"sku":"24F2DUXJD4NRSAVM"},"serviceCode":"AmazonEC2","terms":{"OnDemand":{"24F2DUXJD4NRSAVM.JRTCKXETXF":{"priceDimensions":{"24F2DUXJD4NRSAVM.JRTCK

In [37]:
print(os)

SLES


In [112]:
jItems=json.loads(items)
print(jItems)

TypeError: the JSON object must be str, bytes or bytearray, not 'list'

In [144]:
            response = client.get_products(
                Filters=[
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'location',
                        'Value': location
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'operatingSystem',
                        'Value': os
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'instanceFamily',
                        'Value': instanceFamily
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'currentGeneration',
                        'Value': 'Yes'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'licenseModel',
                        'Value': 'No License required'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'tenancy',
                        'Value': 'Shared'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'preInstalledSw',
                        'Value': 'NA'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'operatingSystem',
                        'Value': 'RHEL'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'instanceType',
                        'Value': 'm5.4xlarge'
                    }
                    
                ],
                ServiceCode='AmazonEC2',
                MaxResults=100
            )

In [145]:
print(response)

{'FormatVersion': 'aws_v1', 'PriceList': ['{"product":{"productFamily":"Compute Instance","attributes":{"enhancedNetworkingSupported":"Yes","memory":"64 GiB","dedicatedEbsThroughput":"2120 Mbps","vcpu":"16","capacitystatus":"Used","locationType":"AWS Region","storage":"EBS only","instanceFamily":"General purpose","operatingSystem":"RHEL","physicalProcessor":"Intel Xeon Platinum 8175","clockSpeed":"2.5 GHz","ecu":"60","networkPerformance":"Up to 10 Gigabit","servicename":"Amazon Elastic Compute Cloud","instanceType":"m5.4xlarge","tenancy":"Shared","usagetype":"BoxUsage:m5.4xlarge","normalizationSizeFactor":"32","processorFeatures":"Intel AVX, Intel AVX2, Intel AVX512, Intel Turbo","servicecode":"AmazonEC2","licenseModel":"No License required","currentGeneration":"Yes","preInstalledSw":"NA","location":"US East (N. Virginia)","processorArchitecture":"64-bit","operation":"RunInstances:0010"},"sku":"3PFRBSXSD3ASHUXZ"},"serviceCode":"AmazonEC2","terms":{"OnDemand":{"3PFRBSXSD3ASHUXZ.JRTCKXET

In [146]:
items=response['PriceList']

In [147]:
print(items)

['{"product":{"productFamily":"Compute Instance","attributes":{"enhancedNetworkingSupported":"Yes","memory":"64 GiB","dedicatedEbsThroughput":"2120 Mbps","vcpu":"16","capacitystatus":"Used","locationType":"AWS Region","storage":"EBS only","instanceFamily":"General purpose","operatingSystem":"RHEL","physicalProcessor":"Intel Xeon Platinum 8175","clockSpeed":"2.5 GHz","ecu":"60","networkPerformance":"Up to 10 Gigabit","servicename":"Amazon Elastic Compute Cloud","instanceType":"m5.4xlarge","tenancy":"Shared","usagetype":"BoxUsage:m5.4xlarge","normalizationSizeFactor":"32","processorFeatures":"Intel AVX, Intel AVX2, Intel AVX512, Intel Turbo","servicecode":"AmazonEC2","licenseModel":"No License required","currentGeneration":"Yes","preInstalledSw":"NA","location":"US East (N. Virginia)","processorArchitecture":"64-bit","operation":"RunInstances:0010"},"sku":"3PFRBSXSD3ASHUXZ"},"serviceCode":"AmazonEC2","terms":{"OnDemand":{"3PFRBSXSD3ASHUXZ.JRTCKXETXF":{"priceDimensions":{"3PFRBSXSD3ASHUXZ

In [148]:
 jItem=json.loads(items[0])

In [149]:
print (jItem)

{'product': {'productFamily': 'Compute Instance', 'attributes': {'enhancedNetworkingSupported': 'Yes', 'memory': '64 GiB', 'dedicatedEbsThroughput': '2120 Mbps', 'vcpu': '16', 'capacitystatus': 'Used', 'locationType': 'AWS Region', 'storage': 'EBS only', 'instanceFamily': 'General purpose', 'operatingSystem': 'RHEL', 'physicalProcessor': 'Intel Xeon Platinum 8175', 'clockSpeed': '2.5 GHz', 'ecu': '60', 'networkPerformance': 'Up to 10 Gigabit', 'servicename': 'Amazon Elastic Compute Cloud', 'instanceType': 'm5.4xlarge', 'tenancy': 'Shared', 'usagetype': 'BoxUsage:m5.4xlarge', 'normalizationSizeFactor': '32', 'processorFeatures': 'Intel AVX, Intel AVX2, Intel AVX512, Intel Turbo', 'servicecode': 'AmazonEC2', 'licenseModel': 'No License required', 'currentGeneration': 'Yes', 'preInstalledSw': 'NA', 'location': 'US East (N. Virginia)', 'processorArchitecture': '64-bit', 'operation': 'RunInstances:0010'}, 'sku': '3PFRBSXSD3ASHUXZ'}, 'serviceCode': 'AmazonEC2', 'terms': {'OnDemand': {'3PFRBS

In [162]:
print(os)

RHEL
